## R-alpha

In [2]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 KB 15.8 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 KB 2.7 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream

In [2]:
api_key = '2VTNvx7z'
username = 'R258447'
pwd = '1499'
token_id = '3URP5C2ZUQSYTSEFELMM2CFJGM'

In [3]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

[I 240423 09:36:11 smartConnect:121] in pool


In [4]:
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

In [5]:
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list)

In [6]:
sws.connect()

[I 240423 09:36:23 data:41] on open
[I 240423 09:36:23 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15149041, 'exchange_timestamp': 1713864983000, 'last_traded_price': 4795945, 'subscription_mode_val': 'LTP'}
[I 240423 09:36:24 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15149405, 'exchange_timestamp': 1713864984000, 'last_traded_price': 4795890, 'subscription_mode_val': 'LTP'}
[I 240423 09:36:24 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15149854, 'exchange_timestamp': 1713864984000, 'last_traded_price': 4795765, 'subscription_mode_val': 'LTP'}
[I 240423 09:36:25 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15150222, 'exchange_timestamp': 1713864985000, 'last_traded_price': 4795790, 'subscription_mode_val': 'LTP'}
[I 240423 09:36:26 data:52] Ticks: {'subscription_mode': 1, 

[I 240423 09:37:59 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15228726, 'exchange_timestamp': 1713865079000, 'last_traded_price': 4796775, 'subscription_mode_val': 'LTP'}
[I 240423 09:38:00 data:52] Ticks: {'subscription_mode': 1, 'exchange_type': 1, 'token': '26009', 'sequence_number': 15229145, 'exchange_timestamp': 1713865080000, 'last_traded_price': 4796415, 'subscription_mode_val': 'LTP'}


In [ ]:
sws.close_connection()